In [1]:
! pip install vaderSentiment
! pip install transformers
! pip install googletrans
! pip install NRCLex

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 9.4MB/s 
     |████████████████████████████████| 890kB 28.1MB/s 
     |████████████████████████████████| 1.1MB 36.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=de2b6a213568f51315ecb8e7db35c7d9a4dac9fb7cdf7717d99217c0c542b768
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 931kB 8.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=5b

In [2]:
import nltk
nltk.download('punkt')
 

from transformers import pipeline

import pandas as pd
from textblob import TextBlob
from googletrans import Translator
from wordcloud import WordCloud
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords

from IPython.display import display, HTML

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
drive_path = '/content/drive/Shared drives/DSSG - Twitter Sentiment Analysis on Pandemic Times/tweets/'

tweets = pd.read_csv(drive_path + "tweets_16-31mai.csv")

tweets.head()
pd.set_option('display.max_colwidth', None)

df = tweets[['text']]
df.head()

,text
0,#coronavirus monta cerco a lisboa... preparem-se. Vai valer tudo
1,"EUA e Brasil, os dois países mais castigados pelo coronavírus, podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível. Votem com responsabilidade! Há linhas que não se pode cruzar!"
2,Quais países estão vencendo o coronavírus https://www.osprofanos.com/quais-paises-estao-vencendo-o-coronavirus/ via @osprofanos
3,"#29Maio | #Coronavirus Relatório da DGS, Direção Geral da Saúde, do dia de hoje #COVID19PT"
4,"Assim como a Fiocruz achou em Niterói, no Rio de Janeiro, pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país. O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise. Conto abaixo."


In [4]:
stopwords = {"coronavirus","covid19", "covid19pt","covid","CoronaVirus","FiqueEmCasa", "fiqueemcasa", "Covid19","StayAtHome",
            "Coronavirus.","COVID2019", "COVID19" "Coronavirus", "STOPCOVID19", "ficaemcasa"}

# Function to clean Portuguese text and tokenize it
def preProcessing(text):
    #text = text.lower() # convert text to lower-case
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
    text = word_tokenize(text) # remove repeated characters
    word_list = [word for word in text if word not in stopwords]
    text = ' '.join(word_list)
    return text


translator = Translator()

# Function to translate the current text to english
def Translation(text):
    if  translator.detect(text) != 'en':
        traducao = translator.translate(text, dest='en')
        return traducao.text
    else:
        return text

In [5]:
df['text_clean'] = df['text'].apply(preProcessing)
df['text_en'] = df['text_clean'].apply(Translation)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
df


,text,text_clean,text_en
0,#coronavirus monta cerco a lisboa... preparem-se. Vai valer tudo,monta cerco a lisboa ... preparem-se . Vai valer tudo,set up siege to lisbon ... get ready. It will be worth everything
1,"EUA e Brasil, os dois países mais castigados pelo coronavírus, podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível. Votem com responsabilidade! Há linhas que não se pode cruzar!","EUA e Brasil , os dois países mais castigados pelo coronavírus , podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível . Votem com responsabilidade ! Há linhas que não se pode cruzar !","USA and Brazil, the two countries most affected by the coronavirus, may be the two main examples of the failure of a system that everyone considered infallible. Vote responsibly! There are lines that cannot be crossed!"
2,Quais países estão vencendo o coronavírus https://www.osprofanos.com/quais-paises-estao-vencendo-o-coronavirus/ via @osprofanos,Quais países estão vencendo o coronavírus via,Which countries are beating the coronavirus via
3,"#29Maio | #Coronavirus Relatório da DGS, Direção Geral da Saúde, do dia de hoje #COVID19PT","29Maio | Coronavirus Relatório da DGS , Direção Geral da Saúde , do dia de hoje COVID19PT","29May | Coronavirus Report from DGS, General Directorate of Health, today COVID19PT"
4,"Assim como a Fiocruz achou em Niterói, no Rio de Janeiro, pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país. O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise. Conto abaixo.","Assim como a Fiocruz achou em Niterói , no Rio de Janeiro , pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país . O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise . Conto abaixo .","Just as Fiocruz found it in Niterói, Rio de Janeiro, researchers in Portugal also identified traces of the new coronavirus in the country's sewage system. The project will last a year and wants to help prevent a new crisis. I count below."
5,Os Grandes Prémios da Grã-Bretanha e da Austrália em Silverstone e em Philip Island em MotoGP foram canceladas devido a um surto de Coronavirus. Mais dois pódios para o COVID. #COVID2019 #STOPCOVID19 #MotoGP #FicaEmCasa #BritishGP #AustralianGP,Os Grandes Prémios da Grã-Bretanha e da Austrália em Silverstone e em Philip Island em MotoGP foram canceladas devido a um surto de Coronavirus . Mais dois pódios para o COVID . MotoGP FicaEmCasa BritishGP AustralianGP,The British and Australian Grand Prix at Silverstone and Philip Island in MotoGP were canceled due to a Coronavirus outbreak. Two more podiums for COVID. MotoGP StayAt Home BritishGP AustralianGP
6,Nunca botei fé nesse banho de 5 minutos. Lave bem as mãos. Mas não pare por aí. Capricha no resto também. #coronavirus #nadanavidaéporacaso #andreadovalle em Boca do Inferno https://www.instagram.com/p/CAw6XKFBYHh/?igshid=xnyll71b5sht,Nunca botei fé nesse banho de 5 minutos . Lave bem as mãos . Mas não pare por aí . Capricha no resto também . nadanavidaéporacaso andreadovalle em Boca do Inferno,I never put faith in that 5 minute bath. Wash your hands thoroughly. But don't stop there. Fancy the rest too. nadanavidaéporacaso andreadovalle in Boca do Inferno
7,Ministra defende respostas sociais para lá do combate ao novo coronavírus http://dlvr.it/RXZ4Wp Em http://www.TSF.pt,Ministra defende respostas sociais para lá do combate ao novo coronavírus Em,Minister defends social responses beyond combating the new coronavirus Em
8,Ministra defende respostas sociais para lá do combate ao coronavírus http://dlvr.it/RXXnbW Em http://www.TSF.pt,Ministra defende respostas sociais para lá do combate ao coronavírus Em,Minister defends social responses beyond the fight against coronavirus Em
9,"#27Maio | #Coronavirus Relatório da DGS, Direção Geral da Saúde, do dia de hoje #COVID19PT","27Maio | Co

# Hugging Face Transformers Sentiment Analysis Pipeline

https://github.com/huggingface/transformers

Pipelines encapsulate the overall process of every NLP process:
 
 1. *Tokenization*: Split the initial input into multiple sub-entities with ... properties (i.e. tokens).
 2. *Inference*: Maps every tokens into a more meaningful representation. 
 3. *Decoding*: Use the above representation to generate and/or extract the final output for the underlying task.


Pipelines do to sentiment analysis: identifying if a sequence is positive or negative. It leverages a fine-tuned model on sst2, which is a GLUE task.


## glue/sst2

Config description: The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

Homepage: https://nlp.stanford.edu/sentiment/index.html

Download size: 7.09 MiB

In [ ]:
# Allocate a pipeline for sentiment-analysis
# nlp = pipeline('sentiment-analysis')

#def getSentiment(sentence):
#  sentiment = nlp(sentence)[0]['label']
#  return sentiment

#def getScore(sentence):
#  score = nlp(sentence)[0]['score']
#  return score

#df['classification'] = df['text_en'].apply(getSentiment)
#df['score'] = df['text_en'].apply(getScore)

In [ ]:
df

,text,text_clean,text_en
0,#coronavirus monta cerco a lisboa... preparem-se. Vai valer tudo,monta cerco a lisboa ... preparem-se . Vai valer tudo,set up siege to lisbon ... get ready. It will be worth everything
1,"EUA e Brasil, os dois países mais castigados pelo coronavírus, podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível. Votem com responsabilidade! Há linhas que não se pode cruzar!","EUA e Brasil , os dois países mais castigados pelo coronavírus , podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível . Votem com responsabilidade ! Há linhas que não se pode cruzar !","USA and Brazil, the two countries most affected by the coronavirus, may be the two main examples of the failure of a system that everyone considered infallible. Vote responsibly! There are lines that cannot be crossed!"
2,Quais países estão vencendo o coronavírus https://www.osprofanos.com/quais-paises-estao-vencendo-o-coronavirus/ via @osprofanos,Quais países estão vencendo o coronavírus via,Which countries are beating the coronavirus via
3,"#29Maio | #Coronavirus Relatório da DGS, Direção Geral da Saúde, do dia de hoje #COVID19PT","29Maio | Coronavirus Relatório da DGS , Direção Geral da Saúde , do dia de hoje COVID19PT","29May | Coronavirus Report from DGS, General Directorate of Health, today COVID19PT"
4,"Assim como a Fiocruz achou em Niterói, no Rio de Janeiro, pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país. O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise. Conto abaixo.","Assim como a Fiocruz achou em Niterói , no Rio de Janeiro , pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país . O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise . Conto abaixo .","Just as Fiocruz found it in Niterói, Rio de Janeiro, researchers in Portugal also identified traces of the new coronavirus in the country's sewage system. The project will last a year and wants to help prevent a new crisis. I count below."
5,Os Grandes Prémios da Grã-Bretanha e da Austrália em Silverstone e em Philip Island em MotoGP foram canceladas devido a um surto de Coronavirus. Mais dois pódios para o COVID. #COVID2019 #STOPCOVID19 #MotoGP #FicaEmCasa #BritishGP #AustralianGP,Os Grandes Prémios da Grã-Bretanha e da Austrália em Silverstone e em Philip Island em MotoGP foram canceladas devido a um surto de Coronavirus . Mais dois pódios para o COVID . MotoGP FicaEmCasa BritishGP AustralianGP,The Grand Prix of Great Britain and Australia at Silverstone and Philip Island in MotoGP were canceled due to an outbreak of Coronavirus. Two more podiums for COVID. MotoGP StayAt Home BritishGP AustralianGP
6,Nunca botei fé nesse banho de 5 minutos. Lave bem as mãos. Mas não pare por aí. Capricha no resto também. #coronavirus #nadanavidaéporacaso #andreadovalle em Boca do Inferno https://www.instagram.com/p/CAw6XKFBYHh/?igshid=xnyll71b5sht,Nunca botei fé nesse banho de 5 minutos . Lave bem as mãos . Mas não pare por aí . Capricha no resto também . nadanavidaéporacaso andreadovalle em Boca do Inferno,I never put faith in that 5 minute bath. Wash your hands thoroughly. But don't stop there. Fancy the rest too. nadanavidaéporacaso andreadovalle in Boca do Inferno
7,Ministra defende respostas sociais para lá do combate ao novo coronavírus http://dlvr.it/RXZ4Wp Em http://www.TSF.pt,Ministra defende respostas sociais para lá do combate ao novo coronavírus Em,Minister defends social responses beyond combating the new coronavirus Em
8,Ministra defende respostas sociais para lá do combate ao coronavírus http://dlvr.it/RXXnbW Em http://www.TSF.pt,Ministra defende respostas sociais para lá do combate ao coronavírus Em,Minister defends social responses beyond the fight against coronavirus Em
9,"#27Maio | #Coronavirus Relatório da DGS, Direção Geral da Saúde, do dia de hoje #COVID19PT",

# Lexicon-based

https://github.com/cjhutto/vaderSentiment

Supports: 
- typical negations (e.g., "not good")
- use of contractions as negations (e.g., "wasn't very good")
conventional use of punctuation to signal increased sentiment intensity (e.g., "Good!!!")
- conventional use of word-shape to signal emphasis (e.g., using ALL CAPS for words/phrases)
- using degree modifiers to alter sentiment intensity (e.g., intensity boosters such as "very" and intensity dampeners such as "kind of")
- understanding many sentiment-laden slang words (e.g., 'sux')
- understanding many sentiment-laden slang words as modifiers such as 'uber' or 'friggin' or 'kinda'
- understanding many sentiment-laden emoticons such as :) and :D
- translating utf-8 encoded emojis such as cupid and kiss and grin
- understanding sentiment-laden initialisms and acronyms (for example: 'lol')

## Scoring 
The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
- negative sentiment: compound score <= -0.05

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def vaderSentimentPos(sentence):
  vs = analyzer.polarity_scores(sentence)
  return vs['pos']

def vaderSentimentCompound(sentence):
  vs = analyzer.polarity_scores(sentence)
  return vs['compound']

def vaderSentimentNeu(sentence):
  vs = analyzer.polarity_scores(sentence)
  return vs['neu']

def vaderSentimentNeg(sentence):
  vs = analyzer.polarity_scores(sentence)
  return vs['neg']

# Function to get classification

def getClassification(compound):
  if compound >= 0.05:
    return 'Positive'
  elif compound > -0.05 and compound < 0.05:
    return 'Neutral'
  elif compound <= -0.05:
    return 'Negative'
        
def pretty_print(df):
  return display(HTML( df.to_html().replace("\\n","<br>")))



df['pos'] = df['text_en'].apply(vaderSentimentPos)
df['neu'] = df['text_en'].apply(vaderSentimentNeu)
df['neg'] = df['text_en'].apply(vaderSentimentNeg)
df['compound'] = df['text_en'].apply(vaderSentimentCompound)

df['Classification'] = df['compound'].apply(getClassification)

df


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,text,text_clean,text_en,pos,neu,neg,compound,Classification
0,#coronavirus monta cerco a lisboa... preparem-se. Vai valer tudo,monta cerco a lisboa ... preparem-se . Vai valer tudo,set up siege to lisbon ... get ready. It will be worth everything,0.286,0.714,0.000,0.5267,Positive
1,"EUA e Brasil, os dois países mais castigados pelo coronavírus, podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível. Votem com responsabilidade! Há linhas que não se pode cruzar!","EUA e Brasil , os dois países mais castigados pelo coronavírus , podem ser os dois principais exemplos da falência de um sistema que todo mundo considerava infalível . Votem com responsabilidade ! Há linhas que não se pode cruzar !","USA and Brazil, the two countries most affected by the coronavirus, may be the two main examples of the failure of a system that everyone considered infallible. Vote responsibly! There are lines that cannot be crossed!",0.000,0.855,0.145,-0.6982,Negative
2,Quais países estão vencendo o coronavírus https://www.osprofanos.com/quais-paises-estao-vencendo-o-coronavirus/ via @osprofanos,Quais países estão vencendo o coronavírus via,Which countries are beating the coronavirus via,0.000,0.667,0.333,-0.4588,Negative
3,"#29Maio | #Coronavirus Relatório da DGS, Direção Geral da Saúde, do dia de hoje #COVID19PT","29Maio | Coronavirus Relatório da DGS , Direção Geral da Saúde , do dia de hoje COVID19PT","29May | Coronavirus Report from DGS, General Directorate of Health, today COVID19PT",0.000,1.000,0.000,0.0000,Neutral
4,"Assim como a Fiocruz achou em Niterói, no Rio de Janeiro, pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país. O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise. Conto abaixo.","Assim como a Fiocruz achou em Niterói , no Rio de Janeiro , pesquisadores em Portugal tb identificaram vestígios do novo coronavírus na rede de esgoto do país . O projeto vai durar um ano e quer ajudar na prevenção de uma nova crise . Conto abaixo .","Just as Fiocruz found it in Niterói, Rio de Janeiro, researchers in Portugal also identified traces of the new coronavirus in the country's sewage system. The project will last a year and wants to help prevent a new crisis. I count below.",0.081,0.832,0.087,-0.3182,Negative
5,Os Grandes Prémios da Grã-Bretanha e da Austrália em Silverstone e em Philip Island em MotoGP foram canceladas devido a um surto de Coronavirus. Mais dois pódios para o COVID. #COVID2019 #STOPCOVID19 #MotoGP #FicaEmCasa #BritishGP #AustralianGP,Os Grandes Prémios da Grã-Bretanha e da Austrália em Silverstone e em Philip Island em MotoGP foram canceladas devido a um surto de Coronavirus . Mais dois pódios para o COVID . MotoGP FicaEmCasa BritishGP AustralianGP,The Grand Prix of Great Britain and Australia at Silverstone and Philip Island in MotoGP were canceled due to an outbreak of Coronavirus. Two more podiums for COVID. MotoGP StayAt Home BritishGP AustralianGP,0.186,0.814,0.000,0.7964,Positive
6,Nunca botei fé nesse banho de 5 minutos. Lave bem as mãos. Mas não pare por aí. Capricha no resto também. #coronavirus #nadanavidaéporacaso #andreadovalle em Boca do Inferno https://www.instagram.com/p/CAw6XKFBYHh/?igshid=xnyll71b5sht,Nunca botei fé nesse banho de 5 minutos . Lave bem as mãos . Mas não pare por aí . Capricha no resto também . nadanavidaéporacaso andreadovalle em Boca do Inferno,I never put faith in that 5 minute bath. Wash your hands thoroughly. But don't stop there. Fancy the rest too. nadanavidaéporacaso andreadovalle in Boca do Inferno,0.093,0.851,0.057,0.2643,Positive
7,Ministra defende respostas sociais para lá do combate ao novo coronavírus http://dlvr.it/RXZ4Wp Em http://www.TSF.pt,Ministra defende respostas sociais para lá do combate ao novo coronavírus Em,Minister defends social responses beyond combating the new coronavirus Em,0.000,1.000,0.000,0.0000,Neutral
8,Ministra defende respostas sociais para lá d

In [ ]:
header = ["text", "Classification", "compound"]

df.to_csv(drive_path + '16-31mai.csv', columns = header)

# Lexicon-diverse

NRCLex

NRCLex will measure emotional affect from a body of text. Affect dictionary contains approximately 27,000 words, and is based on the National Research Council Canada (NRC) affect lexicon (see link below) and the NLTK library's WordNet synonym sets.

Lexicon source is (C) 2016 National Research Council Canada (NRC) and this package is for research purposes only. Source: [lexicons for research] (http://sentiment.nrc.ca/lexicons-for-research/)

NLTK data is (C) 2019, NLTK Project. Source: [NLTK] (https://www.nltk.org/). Reference: Bird, Steven, Edward Loper and Ewan Klein (2009), Natural Language Processing with Python. O’Reilly Media Inc.



In [39]:
from nrclex import NRCLex

text_object = NRCLex('you are so stupid!!!')
print(text_object.top_emotions)

[('fear', 0.1111111111111111), ('anger', 0.1111111111111111), ('trust', 0.1111111111111111), ('surprise', 0.1111111111111111), ('positive', 0.1111111111111111), ('negative', 0.1111111111111111), ('sadness', 0.1111111111111111), ('disgust', 0.1111111111111111), ('joy', 0.1111111111111111)]
